In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Solution
import pandas as pd
import numpy as np
from joblib import dump

In [3]:
X_train = pd.read_csv('../data/processed/X_train.csv')
X_val   = pd.read_csv('../data/processed/X_val.csv'  )
X_test  = pd.read_csv('../data/processed/X_test.csv' )
y_train = pd.read_csv('../data/processed/y_train.csv')
y_val   = pd.read_csv('../data/processed/y_val.csv'  )
y_test  = pd.read_csv('../data/processed/y_test.csv' )

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
poly = PolynomialFeatures(2)

In [6]:
X_train = poly.fit_transform(X_train)

In [7]:
X_train.shape

(66642, 378)

In [8]:
X_val = poly.transform(X_val)
X_test = poly.transform(X_test)

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
reg = LinearRegression()

In [11]:
reg.fit(X_train, y_train)

LinearRegression()

In [12]:
y_train_preds = reg.predict(X_train)
y_val_preds = reg.predict(X_val)

In [13]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [14]:
print(mse(y_train_preds, y_train, squared=False))
print(mae(y_train_preds, y_train))

0.20456209547630802
0.14061291921291347
